In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict

args = read_args(api_key='../keys/production_api_scicrunch_key.txt', db_url='../keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.db_url)

In [25]:
terms = sql.get_terms()

In [26]:
terms

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
0,1,ilx_0100000,32290,0,32290,0,(R)N6-phenylisopropyladenosine,,,term,5,0,1,1481226703,1516219713
1,2,ilx_0100001,32290,0,32290,0,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,,term,5,0,1,1481226705,1516219715
2,3,ilx_0100002,32290,0,32290,0,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,,term,5,0,1,1481226707,1516219719
3,4,ilx_0100003,32290,0,32290,0,1-120 Truncated Alpha-Synuclein Drosophila,This transgenic drosophila expresses a form of...,,term,5,0,1,1481226709,1516736120
4,5,ilx_0100004,32290,0,32290,0,1-D extent,A quality inhering in a bearer by virtue of it...,,term,4,0,1,1481226711,1516219725
5,6,ilx_0100005,32290,0,32290,0,11-dehydro-thromboxane B2,,,term,4,0,1,1481226713,1516219728
6,7,ilx_0100006,32290,0,32290,0,129,,,term,3,0,1,1481226715,1516219731
7,8,ilx_0100007,32290,0,32290,0,1D polyacrylamide gel electrophoresis instrument,,,term,3,0,1,1481226716,1516219735
8,9,ilx_0100008,32290,0,32290,0,1D Time-series analysis software,,,term,3,0,1,1481226718,1516219738
9,10,ilx_0100009,32290,0,32290,0,2-Chloro-Adenosine 5'-diphosphate,,,term,3,0,1,1481226720,1516219741


In [14]:
superclasses = sql.get_superclasses()

In [ ]:
#term_superclasses
# id         tid1      tid2
#[(12461, 15030, 160, 1, 1491829905),
# (12462, 15031, 160, 1, 1491830438),
# (12463, 15032, 5022, 1, 1491831300),
# (147945, 150526, 0, 1, 1515533600),
# (147946, 150527, 0, 1, 1515533600),
# (147951, 150532, 0, 1, 1515534044),
# (147956, 150537, 0, 1, 1515534088),
# (147957, 150538, 0, 1, 1515534088)]

In [46]:
terms.loc[terms.ilx == 'ilx_0232950']

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
134253,134258,ilx_0232950,0,0,0,0,Minimally Verbal - Common Demographic Intake Form,,,cde,2,0,1,1515482174,1519760483
150401,150406,ilx_0232950,0,0,0,0,Minimally Verbal - Common Demographic Intake Form,sources : NDAR; categories : Demographics; dat...,,cde,1,0,1,1515530752,1515530752
150520,150525,ilx_0232950,0,0,0,0,Minimally Verbal - Common Demographic Intake Form,sources : NDAR; categories : Demographics; dat...,,cde,1,0,1,1515533599,1515533599
150534,150539,ilx_0232950,0,0,0,0,Minimally Verbal - Common Demographic Intake Form,sources : NDAR; categories : Demographics; dat...,,cde,1,0,1,1515547511,1515547511


In [54]:
terms.loc[terms.label == 'src_subject_id (Minimally Verbal - Common Demographic Intake Form)']

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
134255,134260,ilx_0232952,0,0,0,0,src_subject_id (Minimally Verbal - Common Demo...,Subject ID how it&#39;s defined in lab/project,,cde,4,0,1,1515482176,1524597784
150403,150408,ilx_0232952,0,0,0,0,src_subject_id (Minimally Verbal - Common Demo...,description : Subject ID how it&#39;s defined ...,,cde,1,0,1,1515530754,1515530754
150521,150526,ilx_0232952,0,0,0,0,src_subject_id (Minimally Verbal - Common Demo...,description : Subject ID how it&#39;s defined ...,,cde,1,0,1,1515533600,1515533600
150536,150541,ilx_0232952,0,0,0,0,src_subject_id (Minimally Verbal - Common Demo...,description : Subject ID how it&#39;s defined ...,,cde,1,0,1,1515547513,1515547513


In [55]:
superclasses.loc[superclasses.tid == 134260]

,id,tid,superclass_tid,version,time
205567,1027935,134260,134258,4,1524597784


In [43]:
superclasses.loc[superclasses.id == 0]

,id,tid,superclass_tid,version,time


In [39]:
superclass_dict = {}
for i, row in superclasses.iterrows():
    superclass_dict.update({str(row.id):row})
    if int(row.superclass_tid) == 0:
        print(i)

6919
6920
6922
6924
6925


In [60]:
360 + 80*2 + 190*2 + 160*2 + 200 + 200 + 100 + 100 + 190

2010

In [58]:
1900 + 150 + 400

2450

In [35]:
for i, row in terms.iterrows():
    try:
        if not superclass_dict.get(str(row['id'])).empty:
            if str(row['type']) == 'cde':
                print(row); break
    except:
        pass

0 term
1 term
2 term
3 term
4 term
5 term
6 term
7 term
8 term
9 term
10 term
11 term
12 term
13 term
14 term
15 term
16 term
17 term
18 term
19 term
20 term
21 term
22 term
23 term
24 term
25 term
26 term
27 term
28 term
29 term
30 term
31 term
32 term
33 term
34 term
35 term
36 term
37 term
38 term
39 term
40 term
41 term
42 term
43 term
44 term
45 term
46 term
47 term
48 term
49 term
50 term
51 term
52 term
53 term
54 term
55 term
56 term
57 term
58 term
59 term
60 term
61 term
62 term
63 term
64 term
65 term
66 term
67 term
68 term
69 term
70 term
71 term
72 term
73 term
74 term
75 term
76 term
77 term
78 term
79 term
80 term
81 term
82 term
83 term
84 term
85 term
86 term
87 term
88 term
89 term
90 term
91 term
92 term
93 term
94 term
95 term
96 term
97 term
98 term
99 term
100 term
101 term
102 term
103 term
104 term
105 term
106 term
107 term
108 term
109 term
110 term
111 term
112 term
113 term
114 term
115 term
116 term
117 term
118 term
119 term
120 term
121 term
122 term
123

In [7]:
tier1 = '/home/troy/Desktop/NDA/nda-cdes-first-tier-only-update-ready.json'
tier2 = '/home/troy/Desktop/NDA/complete_nda_tier2_production_ref.json'
nda_names = list(json.load(open(tier1)).keys()) + list(json.load(open(tier2)).keys())
tier1_data = json.load(open(tier1))
tier2_data = json.load(open(tier2))

In [39]:
labels_to_id = sql.get_labels_to_ids_dict()
terms_df = sql.get_terms()

## Checking if any missing T1 labels

In [40]:
t1_names = [name.lower().replace("'", '').strip() for name in list(tier2_data)] #just ended up getting rid of the ' altogether

for name in t1_names:
    if not labels_to_id.get(name):
        print(name)

In [41]:
def link(ilxid):
    return "https://scicrunch.org/scicrunch/interlex/view/%s?searchTerm=%s"%(ilxid, ilxid)

In [ ]:
link(

In [42]:
data = []
for name in t1_names:
    tid = labels_to_id[name]
    seg_df = terms_df.loc[terms_df.id == tid['cde']]
    if not seg_df.empty:
        if 'modifiedDate' in str(seg_df.definition.iloc[0]):
            print(name, tid)
            print(link(seg_df['ilx'].iloc[0]))
            data.append({'id':tid['cde'], 'definition':' '})
json.dump(data, open('../clean_t1_definitions.json', 'w'), indent=4)

In [4]:
nda_ids = []
for label in nda_names:
    label = label.lower().strip().replace('"', '').replace("'", '')
    if not labels_to_id.get(label):
        sys.exit(label)
    else:
        nda_ids.append(labels_to_id[label]['cde'])

In [33]:
terms_df = sql.get_terms()
annotations_df = sql.get_annotations()

In [6]:
id_to_label = sql.get_ids_to_labels_dict()

In [7]:
annotations_df.loc[annotations_df.item_id == 230000].to_dict('list')

{'item_id': [230000, 230000, 230000, 230000, 230000, 230000, 230000, 230000],
 'label': ['vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)'],
 'ilx': ['ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138'],
 'anno_tid': [15074, 15074, 15074, 15074, 15074, 15074, 15074, 15074],
 'anno_label': ['hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint'],
 'anno_ilx': ['ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071'],
 'tid': [230000, 230000, 230000, 230000, 230000, 230000, 230000, 230000],
 'annotatio

In [23]:
nda_ids[0]['cde']

15076

In [59]:
annotations_df.loc[annotations_df.label == 'eventname (ABCD Children&#39;s Report of Parental Behavioral Inventory)']

,item_id,label,ilx,anno_tid,anno_label,anno_ilx,tid,annotation_tid,value
12891,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,required : Recommended
12892,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,condition : None
12893,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,filterElement : None
12894,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,type : String
12895,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,size : 60
12896,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,valueRange : None
12897,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,notes : None


In [34]:
%%time
cdes_no_annotations = []
for id in nda_ids[:]:
    hit = annotations_df.loc[annotations_df.tid == id]
    if hit.empty:
        cdes_no_annotations.append({'id':id, 'label':id_to_label[id]})
print(len(cdes_no_annotations))
#with open('/home/troy/Desktop/NDA/missing_annotaions.json', 'w') as f:
#json.dump(cdes_no_annotations, f, indent=4)

0
CPU times: user 5min 29s, sys: 20.3 ms, total: 5min 29s
Wall time: 5min 29s


In [67]:
for id in nda_ids:
    if id == 15813:
        print('yas')

In [72]:
#%%time
cdes_incomplete_annotations = []
for id in nda_ids[:]:
    if id != 15813: continue
    print(id)
    hit = annotations_df.loc[annotations_df.tid == id]
    if hit.empty:
        pass#cdes_no_annotations.append({'id':id, 'label':id_to_label[id]})
    else:
        vs = hit.to_dict('list')['value']
        if '(abcd children' in id_to_label[id].lower():
            print(id_to_label[id])
            sys.exit(vs)
        #if 'id' not in ''.join(vs) and '(' in id_to_label[id] and id > 16000:
            #print(vs)
        #if len(hit.to_dict('list')['value']) not in [5, 8]:
        #    cdes_incomplete_annotations.append({'id':id, 'label':id_to_label[id]})
        #    sys.exit(cdes_incomplete_annotations)
hit           
#print(len(cdes_incomplete_annotations))
#with open('/home/troy/Desktop/NDA/incomplete_annotaions.json', 'w') as f:
#    json.dump(cdes_incomplete_annotations, f, indent=4)

[]

In [20]:
missing_annotatons_data = json.load(open('/home/troy/Desktop/NDA/missing_annotaions.json'))
incomplete_annotations_data = json.load(open('/home/troy/Desktop/NDA/incomplete_annotaions.json'))

In [25]:
tier2_data

{'subjectkey (Abnormal Involuntary Movement Scale)': {'label': 'subjectkey (Abnormal Involuntary Movement Scale)',
  'definition': 'The NDAR Global Unique Identifier (GUID) for research subject',
  'type': 'cde',
  'annotation_tid': 15074,
  'value': ['condition : None',
   'filterElement : None',
   'id : 2039689',
   'notes : None',
   'required : Required',
   'size : None',
   'type : GUID',
   'valueRange : NDAR*'],
  'superclass_label': 'Abnormal Involuntary Movement Scale',
  'superclass_ilx': 'ilx_0339882',
  'superclass_tid': 258573},
 'src_subject_id (Abnormal Involuntary Movement Scale)': {'label': 'src_subject_id (Abnormal Involuntary Movement Scale)',
  'definition': "Subject ID how it's defined in lab/project",
  'type': 'cde',
  'annotation_tid': 15074,
  'value': ['condition : None',
   'filterElement : None',
   'id : 2039690',
   'notes : None',
   'required : Required',
   'size : 20',
   'type : String',
   'valueRange : None'],
  'superclass_label': 'Abnormal Invol

In [30]:
''' Fix missing annotations '''
annotations = []
for e in missing_annotatons_data:
    if tier1_data.get(e['label']):
        t1 = tier1_data[e['label']]
        for v in t1['value']:
            annotations.append({'tid':e['id'], 'annotation_tid':15074, 'value':v})
    elif tier2_data.get(e['label']):
        t2 = tier2_data[e['label']]
        for v in t2['value']:
            annotations.append({'tid':e['id'], 'annotation_tid':15074, 'value':v})
    else:
        sys.exit(e)
len(annotations)
with open('/home/troy/Desktop/NDA/missing_annotaions_client_ready.json', 'w') as f:
    json.dump(annotations, f, indent=4)

In [53]:
annotations = []
for e in incomplete_annotations_data:
    
    if tier1_data.get(e['label']):
        t1 = tier1_data[e['label']]
        for v in t1['value']:
            annotations.append({'tid':str(e['id']), 'annotation_tid':15074, 'value':str(v)})
    
    elif tier2_data.get(e['label']):
        t2 = tier2_data[e['label']]
        for v in t2['value']:
            annotations.append({'tid':str(e['id']), 'annotation_tid':15074, 'value':str(v)}) 
        print(e['label'])
    else:
        sys.exit(e)
        
len(annotations)
with open('/home/troy/Desktop/NDA/incomplete_annotaions_client_ready.json', 'w') as f:
    json.dump(annotations, f, indent=4)

bdi1 (Beck Depression Inventory)
bdi4 (Beck Depression Inventory)
bdi5 (Beck Depression Inventory)
bdi6 (Beck Depression Inventory)
bdi8 (Beck Depression Inventory)
bdi9 (Beck Depression Inventory)
bdi10 (Beck Depression Inventory)
bdi11 (Beck Depression Inventory)
bdi12 (Beck Depression Inventory)
bdi14 (Beck Depression Inventory)
bdi15 (Beck Depression Inventory)
bdi16 (Beck Depression Inventory)
bdi19 (Beck Depression Inventory)
bdi_cat (Beck Depression Inventory)
bd_004 (Beck Depression Inventory)
carp15 (25-year Carpenter Prognostic Update)
carp16 (25-year Carpenter Prognostic Update)
carp17 (25-year Carpenter Prognostic Update)
carp18 (25-year Carpenter Prognostic Update)
grade_highed (AASPIRE Health Integration)
aut_posteval_effect (AASPIRE Health Integration)
sup_prepers_comm (AASPIRE Health Integration)
cash_choice_task (ABCD Cash Choice Task)
devhx_2_p_dk (ABCD Developmental History Questionnaire)
devhx_2b_p_dk (ABCD Developmental History Questionnaire)
devhx_3_p (ABCD Develo

In [2]:
## So the shortNames list is the only complete data for some unknown reason at the moment; best to just use that to make pure annotations and update defintions
tier2 = json.load(open('/home/troy/Desktop/NDA/nda_shortName_data.json', 'r'))

In [16]:
complete_nda_tier2_production_ref = defaultdict(dict)
for k, v in tier2.items():
    print(k)
    for d in v['dataElements']:
        print(d)
        annotations = []
        for annotation_key, annotations_values in v.items():
            if isinstance(annotations_values, list) and annotations_values:
                #print(annotation_key, annotations_values)
                value = annotation_key + ' : ' + '; '.join(annotations_values)
            else:
                value = annotation_key + ' : ' + annotations_values
            annotations.append(value)
        complete_nda_tier2_production_ref[k].update({'value':annotations})
        break
    break
complete_nda_tier2_production_ref

12-Item Grit Scale
{'aliases': [], 'condition': None, 'dataElementId': 1, 'description': 'The NDAR Global Unique Identifier (GUID) for research subject', 'filterElement': None, 'id': 2081167, 'name': 'subjectkey', 'notes': None, 'position': 1, 'required': 'Required', 'size': None, 'translations': [], 'type': 'GUID', 'valueRange': 'NDAR*'}


TypeError: sequence item 0: expected str instance, dict found